In [53]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [54]:
#set up chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\steve\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


In [55]:
#visit url
url = "https://terminalwestatl.com/calendar"
request = requests.get(url)
browser.visit(url)

In [56]:
#wait for browser to load calendar
time.sleep(10)

In [58]:
#parse HTML
html = browser.html
soup = bs(html, "html.parser")

In [60]:
#make lists for dataframe
event_name = []
event_desc = []
event_date = []
event_time = []
event_link = []

In [ ]:
#make list of all the event cards
results = soup.find_all('div', class_='c-axs-event-card__wrapper')

In [61]:
#loop through event cards
for result in results:
    title = result.find('div', class_='c-axs-event-card__title c-axs-event-card__title7124').text
    try:
        supporting = result.find('div', class_="c-axs-event-card__supporting-text c-axs-event-card__supporting-text7124").text
        date = result.find('p', class_='c-axs-event-card__date--showtime7124').text
    except:
        supporting = ""
        date = "n/a"
    # link = result.find('div', class_='c-axs-event-card__header c-axs-event-card__header7124')['href']
    if len(supporting) > 0:
        description = f'{title} performing with {supporting}'
    else:
        description = "n/a"

    event_name.append(title)
    event_desc.append(description)
    event_date.append(date)

In [63]:
#make dataframe
df = pd.DataFrame({
    "Event_Name":event_name,
    "Event_Description":event_desc,
    "Event_Date":event_date
})
df.head(5)

,Event_Name,Event_Description,Event_Date
0,Read Southall Band,Read Southall Band performing with Midnight Ri...,"Wed, Sep 8, 2021 8:00 PM"
1,Town Mountain,Town Mountain performing with Smith & Calvert,"Thu, Sep 9, 2021 8:00 PM"
2,Marvel Years & Maddy O'Neal,Marvel Years & Maddy O'Neal performing with Pu...,"Fri, Sep 10, 2021 9:00 PM"
3,Mark Farina,Mark Farina performing with Caleb Wheels,"Sat, Sep 11, 2021 9:30 PM"
4,Emily King,n/a,n/a
